Давайте решим следующую задачу.<br>
Необходимо написать робота, который будет скачивать новости с сайта Лента.Ру и фильтровать их в зависимости от интересов пользователя. От пользователя требуется отмечать интересующие его новости, по которым система будет выделять области его интересов.<br>


Начнем с загрузки новостей. Для этого нам потребуется метод requests.get(url). Библиотека requests предоставляет серьезные возможности для загрузки информации из Интернет. Метод get получает URL стараницы и возвращает ее содержимое. В нашем случае результат будет получаться в формате html. <br>
Загрузим необходимые библиотеки.

In [1]:
import requests # Загрузка новостей с сайта.
from bs4 import BeautifulSoup # Превращалка html в текст.
import re # Регулярные выражения.

Теперь попробуем загрузить страницу новостей.

In [2]:
# Для пробы получаем первую страницу сайта.
requests.get("https://lenta.ru/")

<Response [200]>

Метод <i>requests.get()</i> возвращает объект Response, который содержит большое количество различной информации о загруженной (или незагруженной) странице. В краткой форме отображается только результат выполения запроса. В нашем случае это 200, нет ошибки.<br> 
Посмотрим что результат содержит еще.

In [3]:
%%time
# %%time - Магия Jupyter - замеряет время выполнения ячейки. Должно быть первой строчкой в ячейке.
resp = requests.get("https://lenta.ru/news/2018/08/24/clon/")
print("cookies:", resp.cookies)
print("time to download:", resp.elapsed)
print("page encoding", resp.encoding)
print("Server response: ", resp.status_code)
print("Is everything ok? ", resp.ok)
print("Page's URL: ", resp.url)

cookies: <RequestsCookieJar[<Cookie is_mobile=0 for .lenta.ru/>, <Cookie lid=vAsAAD1gYmJmIvYsATW/HQB= for .lenta.ru/>, <Cookie lids=48202266980FDA75 for .lenta.ru/>]>
time to download: 0:00:00.311823
page encoding utf-8
Server response:  200
Is everything ok?  True
Page's URL:  https://lenta.ru/news/2018/08/24/clon/
CPU times: user 25.5 ms, sys: 11.3 ms, total: 36.8 ms
Wall time: 330 ms


Но самое для нас интересное хранится в поле <i>text</i>, которое содержит собственно текст html-страницы.

In [4]:
#Берем первые 1000 символов новости.
resp.text[:1000]

'<!DOCTYPE html><html lang="ru"><head><title>В Сибири нашли подходящих для клонирования древних животных: Наука: Наука и техника: Lenta.ru</title><meta charset="utf-8" /><meta content="#292929" name="theme-color" /><link href="https://m.lenta.ru/news/2018/08/24/clon/" media="only screen and (max-width: 640px)" rel="alternate" /><link href="https://lenta.ru/rss/google-newsstand/main/" rel="alternate" type="application/rss+xml" /><link href="https://lenta.ru/news/2018/08/24/clon/" rel="canonical" /><link href="/manifest.json" rel="manifest" /><link rel="shortcut icon" type="image/x-icon" href="https://icdn.lenta.ru/favicon.ico" /><link rel="apple-touch-icon" type="image/x-icon" href="https://icdn.lenta.ru/images/icons/icon-256x256.png" size="256x256" /><link rel="apple-touch-icon" type="image/x-icon" href="https://icdn.lenta.ru/images/icons/icon-192x192.png" size="192x192" /><link rel="apple-touch-icon" type="image/x-icon" href="https://icdn.lenta.ru/images/icons/icon-152x152.png" size="

Количество служебной информации в странице явно превышает объем текста новости. У нас есть два пути: либо использовать библиотеку BeautyfulSoup для получения текста статьи, либо получить текст с использованием регулярных выражений.

Опробуем первый путь. Документация на библиотеку BeautyfulSoup находится <a href="https://www.crummy.com/software/BeautifulSoup/bs4/doc/">здесь</a>.

В ячейке ниже мы создаем объект BeautifulSoup, передаем в него текст html-страницы и сообщаем, что разбирать его надо при помощи библиотеки `html5lib`. Далее просим отдать текст страницы без html-тегов.

In [5]:
BeautifulSoup(resp.text, "html5lib").get_text()[:1000]

'В Сибири нашли подходящих для клонирования древних животных: Наука: Наука и техника: Lenta.ru{"@context":"http://schema.org","@type":"NewsArticle","headline":"В Сибири нашли подходящих для клонирования древних животных","description":"Палеонтологи обнаружили в Якутии тушу жеребенка, возраст которой достигает 30-40 тысяч лет, а также останки мамонта с мягкими тканями. Специалисты отмечают хорошее состояние тела лошади, пролежавшей в вечной мерзлоте. Подобные находки имеют большое количество сохранившейся ДНК, которая подходит для клонирования.","name":"В Сибири нашли подходящих для клонирования древних животных","url":"https://lenta.ru/news/2018/08/24/clon/","mainEntityOfPage":{"@type":"WebPage","@id":"https://lenta.ru/news/2018/08/24/clon/"},"associatedMedia":"","thumbnailUrl":"https://icdn.lenta.ru/images/2018/08/24/13/20180824130540171/detail_fb4ef24e26e8448a2a23747f000a4489.jpg","dateCreated":"2018-08-24T15:22:00+03:00","datePublished":"2018-08-24T15:22:00+03:00","dateModified":"20

Да, убрать html-теги получилось. Но их содержимое осталось, в том числе и скрипты.<br>
Опробуем другой путь. Весь текст обычно оформляется тегом параграфа - &lt;p&gt;. Выберем весь текст из этих тегов. Заодно выберем и заголовок статьи, оформленный при помощи \<h1\>.

In [6]:
# Получили объект BeautifulSoup и скормили ему текст страницы.
bs=BeautifulSoup(resp.text, "html5lib") 
# Вот таким образом можно попросить отдать первый тег, отмеченный как h1. Вместо h1 можно написать любой другой тег.
title=bs.h1.text
# Получаем все параграфы (тег p), берем их текст без тегов и склеиваем в один текст.
text=" ".join([p.text for p in bs.find_all("p")])
print(title, "\n-----\n", text)

В Сибири нашли подходящих для клонирования древних животных 
-----
 Фото: AP Российские палеонтологи обнаружили в Якутии тушу жеребенка, возраст которой достигает 30-40 тысяч лет, а также останки мамонта с мягкими тканями. Об этом сообщается в пресс-релизе на Phys.org. Специалисты отмечают хорошее состояние тела лошади, пролежавшей в вечной мерзлоте. Таким образом, находка является потенциально пригодной для клонирования животного. У найденного ископаемого, относящегося к вымершему виду Equus lenensis, сохранились кожа, шерсть, копыта, хвост и внутренние органы. Возраст жеребенка на момент смерти составлял примерно 2-3 месяца. Причиной смерти, вероятно, является попадание в какую-то «ловушку» естественного происхождения, поскольку видимых повреждений на теле не было. У трупа были взяты образцы шерсти и биологических жидкостей для тщательного генетического анализа. По словам исследователей, на данный момент это самые хорошо сохранившиеся из всех останков древних лошадей. В 2015 году в Я

Получилось хорошо. Но опробуем второй путь.<br>
Теперь попробуем использовать регулярные выражения в два шага. На первом мы вырежем только саму новость с ее оформлением используя для этого регулярные выражения (библиотека re). На втором шаге мы используем библиотеку BeautifulSoup для "выкусывания" тегов html.

In [7]:
# Компилируем регулярные выражения для выделения заголовка, маркера начала и конца статьи, удаления скриптов.
findheaders = re.compile("<h1.+?>(.+)</h1>", re.S)
boa = re.compile('<div class="b-text clearfix js-topic__text" itemprop="articleBody">', re.S)
eoa = re.compile('<div class="b-box">\s*?<i>', re.S)
delscript = re.compile("<script.*?>.+?</script>", re.S)

# Получает текст страницы.
art=requests.get("https://lenta.ru/news/2018/08/24/clon/")
# Находим заголовок.
title = findheaders.findall(art.text)[0]
# Выделяем текст новости.
text = eoa.split(boa.split(art.text)[1])
# Иногда новость оканчивается другим набором тегов.
if len(text)==1:
    text = re.split('<div itemprop="author" itemscope=""', text[0])
# Выкусываем скрипты - BeautifulSoup не справляетсяя с ними.
text = " ".join(delscript.split(text[0]))
# Выкусываем остальные теги.
print(BeautifulSoup(title+"\n-----\n"+text, "lxml").get_text())


IndexError: list index out of range

Обратите внимание на этот фрагмент.

<i>... видимых повреждений на теле не было.У трупа были взяты образцы шерсти...</i>

BeautyfulSoup именно "выкусывает" теги, не заменяя их на пробелы. Иногда это можжет приводить к искожению текста из-за "склеивания" слов.

Попробуем вместо этого заменить теги на пробелы при помощи регулярных выражений.

In [8]:
text=re.sub("<.+?>", " ", text)
print(title+"\n-----\n"+text)

<span class="topic-body__title">В Сибири нашли подходящих для клонирования древних животных</span>
-----
Фото: AP Российские палеонтологи обнаружили в Якутии тушу жеребенка, возраст которой достигает 30-40 тысяч лет, а также останки мамонта с мягкими тканями. Об этом сообщается в пресс-релизе на Phys.org. Специалисты отмечают хорошее состояние тела лошади, пролежавшей в вечной мерзлоте. Таким образом, находка является потенциально пригодной для клонирования животного. У найденного ископаемого, относящегося к вымершему виду Equus lenensis, сохранились кожа, шерсть, копыта, хвост и внутренние органы. Возраст жеребенка на момент смерти составлял примерно 2-3 месяца. Причиной смерти, вероятно, является попадание в какую-то «ловушку» естественного происхождения, поскольку видимых повреждений на теле не было. У трупа были взяты образцы шерсти и биологических жидкостей для тщательного генетического анализа. По словам исследователей, на данный момент это самые хорошо сохранившиеся из всех оста

Стало больше пробелов, но зато никто ни с кем не склеился.

Теперь напишем функцию, которая выгружает все новости за сутки. <br>
Обратим внимание, что для сайта Lenta.ru можно написать адрес в формате lenta.ru/ГГГГ/ММ/ДД/ (год, месяц, день) и получить все новости за этот день. Попробуем получить все адреса с такой страницы.

In [9]:
# Идем на страницу, получаем ее текст, отдаем в BeautifulSoup, ищем все теги ссылок - а.
BeautifulSoup(requests.get("http://lenta.ru/2018/08/25/").text, "html5lib").find_all("a")[:20]

[<a class="js-menu-item" href="/" style="color: #ffffff">Главное</a>,
 <a class="js-menu-item" href="/rubrics/russia/" style="color: #ffffff">Россия</a>,
 <a class="js-menu-item" href="/rubrics/world/" style="color: #ffffff">Мир</a>,
 <a class="js-menu-item" href="/rubrics/ussr/" style="color: #ffffff">Бывший СССР</a>,
 <a class="js-menu-item" href="/rubrics/economics/" style="color: #ffffff">Экономика</a>,
 <a class="js-menu-item" href="/rubrics/forces/" style="color: #ffffff">Силовые структуры</a>,
 <a class="js-menu-item" href="/rubrics/science/" style="color: #ffffff">Наука и техника</a>,
 <a class="js-menu-item" href="/rubrics/culture/" style="color: #ffffff">Культура</a>,
 <a class="js-menu-item" href="/rubrics/sport/" style="color: #ffffff">Спорт</a>,
 <a class="js-menu-item" href="/rubrics/media/" style="color: #ffffff">Интернет и СМИ</a>,
 <a class="js-menu-item" href="/rubrics/style/" style="color: #ffffff">Ценности </a>,
 <a class="js-menu-item" href="/rubrics/travel/" style

Кажется, это опять немного не то, что нам нужно. Мы получили все ссылки, находящиеся на боковом меню, ссылки на события сегодняшнего дня и другие ненужные нам вещи. <br>
Смотрим в содержимое html-страницы и обращаем внимание, что все интересные нам ссылки оформлены как заголовки третьего уровня - &lt;h3&gt;. Извлечем все такие фрагменты, а потом извлечем собственно адреса, помеченные атрибутом href тега &lt;a&gt;.

In [10]:
# Теперь выделим только то, что взято в тег h3.
h3s=BeautifulSoup(requests.get("http://lenta.ru/2018/08/25/").text, "html5lib").find_all("h3")
# Формируем список ссылок. Для этого берем первую (кстати, единственную) ссылку из каждого выделенного
# фрагмента, у нее берем значение параметра href. Так как ссылки внутренние, добавляем к ним адрес сайта.
links=["https://lenta.ru"+l.find_all("a")[0]["href"] for l in h3s]
print(links)

IndexError: list index out of range

Если теперь написать функцию, которая будет перебирать все адреса и получать из них тексты новостей, то мы получим все новости за определенные сутки. Но это мы сделаем чть позже, а пока просто оформим код загрузки статьи в виде функции.

In [12]:
# Загрузка статьи по URL.
def getOneLentaArticle(url):
    """ getLentaArticle gets the body of an article from Lenta.ru"""
    # Получает текст страницы.
    resp=requests.get(url)
    # Загружаем текст в объект типа BeautifulSoup.
    bs=BeautifulSoup(resp.text, "html5lib") 
    # Получаем заголовок статьи.
    aTitle=bs.h1.text.replace("\xa0", " ")
    # Получаем текст статьи.
    anArticle=BeautifulSoup(" ".join([p.text for p in bs.find_all("p")]), "html5lib").get_text().replace("\xa0", " ")
    # Возвращаем кортеж из заголовка и текста статьи.
    return aTitle, anArticle


### XPath
[Ссылка 1](https://habr.com/ru/post/526774/)

[Ссылка 2](https://habr.com/ru/post/464897/)

XPath позволяет задать шаблон для пути от корня XML-дерева к интересующей нас вершине.

- . - корень XML-дерева
- / - переход на один уровень ниже.
- // - переход на ноль или больше уровней вниз.
- \* - любая вершина.
- xyz - название вершины.
- [@feature] - вершина с параметром feature.
- [@feature='111'] - вершина с параметром feature, равным "111".
- xyz[n] - n-ый потомок вершины xyz.

А теперь давайте посмотрим как мы можем при помощи XPath обрабатывать HTML-документы.

In [11]:
from lxml import html

In [12]:
page = requests.get('https://lenta.ru/news/2021/02/27/apple_effect/')

In [24]:
tree = html.fromstring(page.text)
print(tree.xpath(".//h1")[0].text_content())
print(tree.xpath(".//time[contains(@class, 'topic-header__time')]")[0].text_content().strip())
print(tree.xpath(".//div[contains(@class, 'topic-authors')]")[0].text_content().strip(), '\n')
print(tree.xpath(".//meta[@property='og:description']")[0].get("content"), '\n')

for p in tree.xpath(".//div[@itemprop='articleBody']/p"):
    print(p.text_content())

Обнаружен неожиданный эффект от употребления яблок
15:35, 27 февраля 2021
Соня Кошечкина 

Ученые из Университета Квинсленда и Немецкого центра нейродегенеративных заболеваний  обнаружили неожиданный эффект от употребления яблок. Опыты проводились на мышах. Специалисты культивировали стволовые клетки мозга взрослых мышей и добавляли в них содержащиеся в яблоках фитонутриенты. 



А теперь та же страница, но через BeautyfulSoup.

In [25]:
souped = BeautifulSoup(page.text)

title = souped("h1")[0].get_text()
when = souped.find_all("time", attrs={'class': 'topic-header__time'})[0].get_text().strip()
author = souped.find_all("div", attrs={'class': 'topic-authors'})[0].get_text()
description = souped.find_all("meta", attrs={'property': 'og:description'})[0]["content"]

for p in souped.find_all("div", attrs={'class': 'topic-body'})[0]("p"): 
    print(p.get_text())

Анатолий Жданов / «Коммерсантъ»
Ученые из Университета Квинсленда и Немецкого центра нейродегенеративных заболеваний обнаружили неожиданный эффект от употребления яблок. Результаты исследования появились в научном журнале Stem Cell Reports.
Опыты проводились на мышах. Специалисты культивировали стволовые клетки мозга взрослых мышей и добавляли в них содержащиеся в яблоках фитонутриенты. Исследование показало, что высокая концентрация фитонутриентов способствует образованию новых нейронов.
По словам ученых, определенные фитонутриенты положительно влияют на работу органов, в том числе мозга. Выяснилось, что они оказывают на организм тот же эффект, что и физическая активность, которая также стимулирует нейрогенез.
Ранее ученые из Технологического университета австрийского Граца выяснили, что большинство людей неправильно едят яблоки. Исследователи утверждают, что до 90 процентов полезных веществ сосредоточены в сердцевине этого фрукта, и поэтому яблоко желательно съедать вместе с огрызком